In [1]:
import os
import warnings
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
#os.environ['CUDA_LAUNCH_BLOCKING'] = str(1)
#os.environ["TORCH_USE_CUDA_DSA"]= str(0)
warnings.filterwarnings('ignore') 


In [2]:
import copy
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset
import torch.optim as optim
from torch.autograd import Variable
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import math
from collections import OrderedDict
import random
from torchsummary import summary
from torchvision import transforms
from torch.utils.data import Dataset
import sys
import torch
import numpy as np
from tqdm import trange
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mne
from sklearn.preprocessing import StandardScaler
import logging
from torch.nn.utils.rnn import pack_sequence, pad_packed_sequence
import statistics
import torch.optim.lr_scheduler as lr_scheduler
from scipy.special import softmax
from sklearn.model_selection import StratifiedKFold

In [3]:
logging.getLogger('mne').setLevel(logging.WARNING)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import import_ipynb
#from Model import net
from Dataloader2 import EEGDataset

importing Jupyter notebook from Dataloader2.ipynb


In [4]:
def Accuracy(y_pred, y, train_count):
    max_values, _ = torch.max(y_pred, dim=1, keepdim=True)
    mask = y_pred == max_values
    y_pred = mask.int()
    correct_num = torch.sum(torch.all(torch.eq(y, y_pred), dim=1)).item()
    accuracy = correct_num / train_count
    return accuracy * 100

In [5]:
def test(model, test_path, test_class, verbose=True):
    

    x_test = mne.read_epochs(test_path, preload=False).get_data(picks='eeg');
    normals = []
    scaler = StandardScaler()
    for idx in range(len(x_test)):
        normals.append(scaler.fit_transform(x_test[idx]))
    normals = torch.tensor(normals).cuda().float()
    result = torch.argmax(model(normals), axis=1)
    unique_elements, counts = torch.unique(result, return_counts=True)

    votes = np.zeros([4])
    for i in range(len(unique_elements)):
        votes[unique_elements[i]] = counts[i]


    if(verbose):
        print(f"Test Accuracy: {(votes[test_class] / result.shape[0]) * 100}")
    return votes


In [6]:
class net(nn.Module):
    def __init__(self, T, C, input_size, hidden_size, num_layers, spatial_num, dropout, pool):
        super(net, self).__init__()
        
        self.T = T
        self.C = C
        self.spatial_num = spatial_num
        self.dropout = dropout
        self.pool = pool

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.cell_count = self.T // self.input_size

        self.fcn_in = (spatial_num * self.hidden_size)

        self._lstm = nn.LSTM(self.input_size, self.hidden_size, self.num_layers, batch_first=True)

        self.lstm = nn.ModuleList([self._lstm for i in range(self.C)])

        self.cnn_block = nn.Sequential(nn.Conv2d(1, self.spatial_num, (self.C, 1)),
                                       nn.BatchNorm2d(self.spatial_num),
                                       nn.ELU(),



                                       nn.Dropout(self.dropout))

        
        self.fcn = nn.Sequential(nn.Linear(self.fcn_in, 128), 
                                 nn.ReLU(),
                                 nn.Linear(128, 16),
                                 nn.Dropout(self.dropout),
                                 nn.ReLU(),
                                 nn.Linear(16, 4))

        #self.fcn = nn.Linear(self.fcn_in, 4)
        self.results = nn.Softmax(dim=1)
    def forward(self, x):
        x = x.reshape(-1, 1, 19, 3000)
        self.N = x.shape[0]
        x = x.reshape(self.N, self.C, self.cell_count, self.input_size)
        _x = None

        for index, cell in enumerate(self.lstm):
            cell_out, _ = cell(x[:, index, :, :], None)
            last_layer_out = cell_out[:, -1, :]
            
            last_layer_out = last_layer_out.unsqueeze(0)
            if _x is None:
                _x = last_layer_out
            else:
                _x = torch.cat((_x, last_layer_out), dim=0)
            

        x = _x.permute(1, 0, 2).unsqueeze(1)

        x = self.cnn_block(x)


        x = x.reshape(self.N, -1)

        x = self.fcn(x)
        x = self.results(x)

        return x

In [7]:
class CNN_LSTM(nn.Module):
    def __init__(self, num_classes, hidden_size):
        super(CNN_LSTM, self).__init__()
        
        self.num_classes = num_classes
        self.hidden_size =hidden_size
        self.dropout = 0.5
        # CNN layers
        self.conv1 = nn.Conv1d(19, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool1d(2)
        
        # LSTM layers
        self.lstm = nn.LSTM(3000, self.hidden_size, num_layers=1, batch_first=True)
        
        # Fully connected layer
        self.fc = nn.Sequential(nn.Linear(self.hidden_size * 128, 128), 
                                 nn.ReLU(),
                                 nn.Linear(128, 16),
                                 nn.Dropout(self.dropout),
                                 nn.ReLU(),
                                 nn.Linear(16, self.num_classes))
        

        self.results = nn.Softmax(dim=1)
        
        
    def forward(self, x):
        x = x.reshape(-1, 19, 3000)
        self.N = x.shape[0]
        # Apply CNN
        x = F.relu(self.conv1(x))
        #x = self.pool(x)
        
        
        x = F.relu(self.conv2(x))
        #x = self.pool(x)
        
        x = F.relu(self.conv3(x))
        # Reshape for LSTM
        
        #x = x.transpose(1, 2)
        xx = x
        # Apply LSTM
        h_n, (__, _) = self.lstm(x)

        x = h_n[:, :, :]
        
        # Apply fully connected layer
        x = x.reshape(self.N, -1)
        x = self.fc(x)
        x = self.results(x)
        """
        """
        return x
    
model = CNN_LSTM(num_classes=4, hidden_size=100).cuda()
model(torch.rand(7, 19, 3000).cuda()).shape

torch.Size([7, 4])

In [8]:
MNE_Data = EEGDataset(root_dir=r"C:\Users\admin\Desktop\MNE_Data")
labels = MNE_Data.labels
#MNE_Data = EEGDataset(root_dir=r"D:\TEST MNE")
test_path = r"C:\Users\admin\Desktop\TEST\amirifateme.fif"
test_class = 0

In [9]:
train_dataloader = DataLoader(MNE_Data, batch_size=1, shuffle=True)

In [10]:
model = net(T = 3000, C = 19, input_size = 3000, hidden_size = 100, num_layers=1, spatial_num= 300, dropout=0.5, pool=1).to(device)
#model = CNN_LSTM(num_classes=4, hidden_size=100).cuda()
num_params = sum(p.numel() for p in model.parameters())
print("Number of parameters:", num_params)

Number of parameters: 5089660


In [11]:
criterion = nn.CrossEntropyLoss(weight = torch.Tensor([5.3125, 1.8333, 3.5417, 6.6667]).cuda())

optimizer = optim.Adam(model.parameters(), lr=0.001)
#optimizer = optim.Adagrad(model.parameters(), lr=0.001)


scheduler = lr_scheduler.ExponentialLR(optimizer, gamma=0.1)
epochs = 40
kf = StratifiedKFold(n_splits=5)
model_list = []

In [12]:
for train_indices, val_indices in kf.split(MNE_Data, labels):
    train_dataset = torch.utils.data.Subset(MNE_Data, train_indices)
    val_dataset = torch.utils.data.Subset(MNE_Data, val_indices)

    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=1, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=False)

    test_log = []
    log = []


    model = net(T = 3000, C = 19, input_size = 3000, hidden_size = 100, num_layers=1, spatial_num= 300, dropout=0.5, pool=1).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in trange(epochs):
        
        model.train()   
        for index, data in enumerate(train_dataloader):
                
                x, y = data
                y = y.to(torch.float64)
                x = x.reshape(-1, 1, 19, 3000).float()
                x = x[torch.randperm(x.shape[0])]
                y = F.one_hot(torch.tensor(torch.tensor([y.item()]).to(torch.int64)), num_classes=4).expand(x.shape[0], -1).float()
                
                train_count = x.shape[0]
                x, y = x.to(device), y.to(device)
                y_pred = model(x)
                loss = criterion(y_pred, y)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                #scheduler.step()
                torch.cuda.empty_cache()

    
        model.eval()
        val_loss = 0
        val_log = []
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_dataloader):
                        data = data.reshape(-1, 1, 19, 3000).float().cuda()
                        target = F.one_hot(torch.tensor(torch.tensor([target.item()]).to(torch.int64)), num_classes=4).expand(data.shape[0], -1).float().cuda()
                        

                        output = model(data)

                        

                        loss = criterion(output, target)
                        
                        val_loss += loss.item()

                        val_log.append(Accuracy(output, target, target.shape[0]))

        print(model(x.cuda())[-1])     
        print(y[-1])           
        print('Epoch: {}, Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%'.format(
                    epoch, val_loss / len(val_dataloader), statistics.mean(val_log)))

    model_list.append(model)
        


    


  2%|▎         | 1/40 [00:47<30:38, 47.14s/it]

tensor([5.4358e-06, 9.9991e-01, 8.0506e-05, 8.0854e-06], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([0., 0., 1., 0.], device='cuda:0')
Epoch: 0, Validation Loss: 5.9834, Validation Accuracy: 35.29%


  5%|▌         | 2/40 [01:34<29:46, 47.00s/it]

tensor([0.2712, 0.2465, 0.2343, 0.2480], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([0., 1., 0., 0.], device='cuda:0')
Epoch: 1, Validation Loss: 5.2731, Validation Accuracy: 17.65%


  8%|▊         | 3/40 [02:20<28:53, 46.86s/it]

...

tensor([1.0074e-24, 1.0541e-17, 1.7711e-13, 1.0000e+00], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([0., 0., 0., 1.], device='cuda:0')
Epoch: 30, Validation Loss: 5.3105, Validation Accuracy: 27.09%


 80%|████████  | 32/40 [24:33<08:05, 60.73s/it]

tensor([0.6245, 0.1306, 0.1194, 0.1254], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([0., 0., 0., 1.], device='cuda:0')
Epoch: 31, Validation Loss: 5.2957, Validation Accuracy: 27.49%


 82%|████████▎ | 33/40 [25:51<07:41, 65.87s/it]

tensor([1.0000e+00, 6.1547e-08, 1.1206e-10, 1.2892e-10], device='cuda:0',
       grad_fn=<SelectBackward0>)
tensor([1., 0., 0., 0.], device='cuda:0')
Epoch: 32, Validation Loss: 5.3362, Validation Accuracy: 26.25%


In [ ]:
state_dict = model.state_dict()
avg_state_dict = state_dict
for key in state_dict:
    avg_state_dict[key] += state_dict[key]
    print(state_dict[key].shape)
    print('lol')

torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size([400, 3000])
lol
torch.Size([400, 100])
lol
torch.Size([400])
lol
torch.Size([400])
lol
torch.Size

In [ ]:
test(test_path=r"C:\Users\admin\Desktop\TEST\FOCAL\aliabdollahi.fif", model=model, test_class= 3)

FileNotFoundError: File does not exist: "C:\Users\admin\Desktop\TEST\FOCAL\aliabdollahi.fif"

In [ ]:
file_path = 'D:\model weights\model_weights.pth'
torch.save(model.state_dict(), file_path)
model.load_state_dict(torch.load(file_path))

<All keys matched successfully>